In [4]:
!pip install serpapi langchain_community wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=8b2e3d36a4a78203206396413cf4a2f1f4bb256597f59287cb8cad7e1442f47b
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [3]:
!pip install google-search-results

In [58]:
# Import necessary libraries
import os
from serpapi import GoogleSearch
import requests
from openai import ChatCompletion
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
import warnings

warnings.filterwarnings("ignore")

# Set your API keys
os.environ["OPENAI_API_KEY"] = "***************"  
google_maps_api_key = "**************"
weather_api_key = "************"
serpapi_key = "************"


# Step 1: Function to get latitude and longitude for a given location
def get_lat_lng(location):
    """Fetch latitude and longitude for a given location."""
    url = f"https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location,
        "key": google_maps_api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get("results")
        if results:
            location_data = results[0]["geometry"]["location"]
            return location_data["lat"], location_data["lng"]
    return None, None

# Step 2: Function to fetch hotels near a location
def fetch_hotels(location, top_n=5, max_price=None):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    params = {
        "engine": "google_maps",
        "ll": f"@{lat},{lng},14z",
        "q": "hotels",
        "radius": 50000,  # Search within a 50km radius
        "hl": "en",
        "api_key": serpapi_key
    }
    search = GoogleSearch(params)
    results = search.get_dict()

    if "local_results" in results:
        hotels = results["local_results"]
        result = []
        for hotel in hotels[:top_n]:
            name = hotel.get("title", "Name not available")
            rating = hotel.get("rating", "No rating")
            address = hotel.get("address", "Address not available")
            price = hotel.get("price", "Price not available")
            description = hotel.get("description", "Description not available")
            result.append(
                f"Hotel: {name}\nRating: {rating}\nAddress: {address}\nPrice: {price}\nDescription: {description}\n"
                "----------------------------------------"
            )
        return "\n\n".join(result)
    return "No hotel data found in the specified area."

# Function to fetch tourist places
def fetch_tourist_places(location, lat, lng, radius=50000, top_n=5):
    params = {
        "engine": "google_maps",
        "q": "tourist attractions",
        "ll": f"@{lat},{lng},15z",
        "type": "search",
        "radius": radius,
        "hl": "en",
        "api_key": serpapi_key
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    if "local_results" in results and results["local_results"]:
        places = results["local_results"]
        result = []
        for place in places[:top_n]:
            name = place.get("title", "Name not available")
            rating = place.get("rating", "No rating")
            address = place.get("address", "Address not available")
            description = place.get("description", "Description not available")
            result.append(
                f"Place: {name}\nRating: {rating}\nAddress: {address}\nDescription: {description}\n"
                "----------------------------------------"
            )
        return "\n\n".join(result)
    else:
        return "No tourist places found in the specified area."


# Step 4: Function to fetch brief historical summaries for tourist places
def fetch_place_brief_summary(place_name):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{place_name.replace(' ', '_')}"
    response = requests.get(url)
    if response.status_code == 200 and 'extract' in response.json():
        summary = response.json()["extract"].split(". ")
        brief_summary = ". ".join(summary[:2]) + "."  # Limit to 1-2 sentences
        return brief_summary
    return f"No brief information available for {place_name}."

# Step 5: Function to fetch weather information
def fetch_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": weather_api_key,
        "units": "metric"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        weather_data = response.json()
        weather_description = weather_data["weather"][0]["description"]
        temp = weather_data["main"]["temp"]
        return f"The current weather in {city} is {weather_description} with a temperature of {temp}°C."
    return f"Weather data not found for {city}."

# Define the tools for each task
hotel_search_tool = Tool(
    name="Hotel Search",
    func=lambda location, max_price=None: fetch_hotels(location, max_price=max_price),
    description="Searches for hotels in a specified location within a given budget."
)

tourist_places_tool = Tool(
    name="Tourist Places",
    func=lambda location: fetch_tourist_places(location, *get_lat_lng(location)),
    description="Fetches popular tourist places in a specified location."
)


wikipedia_tool = Tool(
    name="Wikipedia History",
    func=fetch_place_brief_summary,
    description="Fetches a brief history or summary of a tourist place."
)

weather_tool = Tool(
    name="Weather",
    func=fetch_weather,
    description="Provides current weather information for a specified city."
)


# Initialize the memory buffer
memory = ConversationBufferMemory()

# Define the prompt template
template = """Give the recommendations for the destination location: {query}"""
prompt = PromptTemplate(
    input_variables=["query"],
    template=template,
)

# Import ChatOpenAI from langchain
from langchain.chat_models import ChatOpenAI

# Initialize the language model using ChatOpenAI from LangChain
llm = ChatOpenAI(model="gpt-4", openai_api_key=os.environ["OPENAI_API_KEY"])

# Initialize the agent with the corrected llm initialization
agent = initialize_agent(
    tools=[hotel_search_tool, tourist_places_tool, wikipedia_tool, weather_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    prompt=prompt,
)


query = input("Enter your query: ")
response = agent.run(query)
print(response)

# Example user query
# Can you recommend some hotels, tourist places, their history, and current weather in hyderabad


Enter your query: recommend me best hotels in boston below 200$ per day


> Entering new AgentExecutor chain...
I need to search for hotels in Boston that cost less than $200 per day.
Action: Hotel Search
Action Input: ("boston", 200)
Observation: Could not determine the exact location.
Thought:It seems there was an issue with the location input, let's try again.
Action: Hotel Search
Action Input: ("Boston, USA", 200)
Observation: Hotel: Hilton Boston Logan Airport
Rating: 4.1
Address: One Hotel Dr, Boston, MA 02128
Price: $184
Description: Upscale airport hotel with a pool. Polished lodging with a skybridge to the airport, plus a pub-style restaurant & an indoor pool.
----------------------------------------

Hotel: Embassy Suites by Hilton Boston at Logan Airport
Rating: 4.1
Address: 207 Porter St, Boston, MA 02128
Price: $174
Description: Warm all-suite hotel with free breakfast. Modern suites in a laid-back lodging with an indoor pool, free breakfast & an airport shuttle.
---------